In [1]:
%config Completer.use_jedi = False
import pandas as pd
from sqlalchemy import create_engine
from config import *

In [2]:
#read raw data
df_raw = pd.read_csv("Resources/netflix_titles.csv")

df_raw.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [21]:
#focus only on 'TV Show's
df_divd = df_raw.loc[df_raw['type'] == 'TV Show']

df_divd.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
5,s6,TV Show,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",Turkey,"July 1, 2017",2016,TV-MA,1 Season,"International TV Shows, TV Dramas, TV Mysteries",A genetics professor experiments with a treatm...
11,s12,TV Show,1983,NaN,"Robert Więckiewicz, Maciej Musiał, Michalina O...","Poland, United States","November 30, 2018",2018,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Dramas","In this dark alt-history thriller, a naïve law..."
12,s13,TV Show,1994,Diego Enrique Osorno,NaN,Mexico,"May 17, 2019",2019,TV-MA,1 Season,"Crime TV Shows, Docuseries, International TV S...",Archival video and new interviews examine Mexi...
16,s17,TV Show,Feb-09,NaN,"Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-...",NaN,"March 20, 2019",2018,TV-14,1 Season,"International TV Shows, TV Dramas","As a psychology professor faces Alzheimer's, h..."


In [22]:
#select only interested data to match to tables in database
df_filtered = df_divd[['title','country','date_added','release_year','description']].copy()

df_filtered

,title,country,date_added,release_year,description
0,3%,Brazil,"August 14, 2020",2020,In a future where the elite inhabit an island ...
5,46,Turkey,"July 1, 2017",2016,A genetics professor experiments with a treatm...
11,1983,"Poland, United States","November 30, 2018",2018,"In this dark alt-history thriller, a naïve law..."
12,1994,Mexico,"May 17, 2019",2019,Archival video and new interviews examine Mexi...
16,Feb-09,NaN,"March 20, 2019",2018,"As a psychology professor faces Alzheimer's, h..."
...,...,...,...,...,...
7767,Zindagi Gulzar Hai,Pakistan,"December 15, 2016",2012,"Strong-willed, middle-class Kashaf and carefre..."
7775,Zoids Wild,Japan,"August 14, 2020",2018,A quest for freedom and legendary treasure beg...
7777,Zombie Dumb,NaN,"July 1, 2019",2018,"While living alone in a spooky town, a young g..."
7779,Zona Rosa,Mexico,"November 26, 2019",2019,An assortment of talent takes the stage for a ...


In [44]:
#check data 
df_filtered.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2410 entries, 0 to 7785
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         2410 non-null   object
 1   country       2133 non-null   object
 2   date_added    2400 non-null   object
 3   release_year  2410 non-null   int64 
 4   description   2410 non-null   object
dtypes: int64(1), object(4)
memory usage: 113.0+ KB


In [45]:
#drop nan/null values. Used dataframe[pd.isnull(dataframe['column']) checked, 
#same results as dataframe.dropna(), 277 nan in 'country' and 10 nan in 'date_added', where there is one occured in both.
df = df_filtered.dropna()

df

,title,country,date_added,release_year,description
0,3%,Brazil,"August 14, 2020",2020,In a future where the elite inhabit an island ...
5,46,Turkey,"July 1, 2017",2016,A genetics professor experiments with a treatm...
11,1983,"Poland, United States","November 30, 2018",2018,"In this dark alt-history thriller, a naïve law..."
12,1994,Mexico,"May 17, 2019",2019,Archival video and new interviews examine Mexi...
24,​SAINT SEIYA: Knights of the Zodiac,Japan,"January 23, 2020",2020,Seiya and the Knights of the Zodiac rise again...
...,...,...,...,...,...
7766,Zig & Sharko,France,"December 1, 2017",2016,"Zig, an island-bound hyena, will do anything t..."
7767,Zindagi Gulzar Hai,Pakistan,"December 15, 2016",2012,"Strong-willed, middle-class Kashaf and carefre..."
7775,Zoids Wild,Japan,"August 14, 2020",2018,A quest for freedom and legendary treasure beg...
7779,Zona Rosa,Mexico,"November 26, 2019",2019,An assortment of talent takes the stage for a ...


In [64]:
#rename the table columns to match with the database, and reset index
df = df.rename(columns={'date_added':'publish_date'})

df1 = df.reset_index(drop=True)

df1

,title,country,publish_date,release_year,description
0,3%,Brazil,"August 14, 2020",2020,In a future where the elite inhabit an island ...
1,46,Turkey,"July 1, 2017",2016,A genetics professor experiments with a treatm...
2,1983,"Poland, United States","November 30, 2018",2018,"In this dark alt-history thriller, a naïve law..."
3,1994,Mexico,"May 17, 2019",2019,Archival video and new interviews examine Mexi...
4,​SAINT SEIYA: Knights of the Zodiac,Japan,"January 23, 2020",2020,Seiya and the Knights of the Zodiac rise again...
...,...,...,...,...,...
2119,Zig & Sharko,France,"December 1, 2017",2016,"Zig, an island-bound hyena, will do anything t..."
2120,Zindagi Gulzar Hai,Pakistan,"December 15, 2016",2012,"Strong-willed, middle-class Kashaf and carefre..."
2121,Zoids Wild,Japan,"August 14, 2020",2018,A quest for freedom and legendary treasure beg...
2122,Zona Rosa,Mexico,"November 26, 2019",2019,An assortment of talent takes the stage for a ...


In [66]:
#read 2nd raw data
dfa = pd.read_csv("Resources/tv_shows_data.csv")

dfa

,Title,Genre,Premiere,No_of_Seasons,No_of_Episodes
0,Stranger Things,Science Fiction Horror,"July 15, 2016",3,25
1,The Crown,Historical Drama,"November 4, 2016",4,40
2,Ozark,Crime Drama,"July 21, 2017",3,30
3,Lost in Space,Science Fiction,"April 13, 2018",2,20
4,Narcos: Mexico,Crime Drama,"November 16, 2018",2,20
...,...,...,...,...,...
636,The Last Narc,Drug Documentary,"July 31, 2020",1,4
637,All or Nothing: Tottenham Hotspur,Sports Documentary,"August 31, 2020",1,9
638,Fernando,Sports Documentary,"September 25, 2020",1,5
639,El Desafío: ETA,Docuseries,"October 30, 2020",1,8


In [67]:
#drop unwanted column
dfa = dfa.drop(['Premiere'],axis=1)

dfa

,Title,Genre,No_of_Seasons,No_of_Episodes
0,Stranger Things,Science Fiction Horror,3,25
1,The Crown,Historical Drama,4,40
2,Ozark,Crime Drama,3,30
3,Lost in Space,Science Fiction,2,20
4,Narcos: Mexico,Crime Drama,2,20
...,...,...,...,...
636,The Last Narc,Drug Documentary,1,4
637,All or Nothing: Tottenham Hotspur,Sports Documentary,1,9
638,Fernando,Sports Documentary,1,5
639,El Desafío: ETA,Docuseries,1,8


In [78]:
#check data
dfa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 640 entries, 0 to 640
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Title           640 non-null    object
 1   Genre           640 non-null    object
 2   No_of_Seasons   640 non-null    object
 3   No_of_Episodes  640 non-null    object
dtypes: object(4)
memory usage: 25.0+ KB


In [79]:
#clean up data, note that there is only one null data but there are invalied strings such as 'Awaiting R/release', 'unknown'
# and 2 rows are duplicate as the column titles
dfa = dfa.dropna()



In [89]:
dfb = dfa.loc[(dfa['Title'] != 'Title') & (dfa['No_of_Episodes'] != 'unknown')]

dfb

,Title,Genre,No_of_Seasons,No_of_Episodes
0,Stranger Things,Science Fiction Horror,3,25
1,The Crown,Historical Drama,4,40
2,Ozark,Crime Drama,3,30
3,Lost in Space,Science Fiction,2,20
4,Narcos: Mexico,Crime Drama,2,20
...,...,...,...,...
636,The Last Narc,Drug Documentary,1,4
637,All or Nothing: Tottenham Hotspur,Sports Documentary,1,9
638,Fernando,Sports Documentary,1,5
639,El Desafío: ETA,Docuseries,1,8


In [93]:
#change columns name to match in database and reset index
dfb.columns = dfb.columns.str.lower()

df2 = dfb.reset_index(drop=True)

df2

,title,genre,no_of_seasons,no_of_episodes
0,Stranger Things,Science Fiction Horror,3,25
1,The Crown,Historical Drama,4,40
2,Ozark,Crime Drama,3,30
3,Lost in Space,Science Fiction,2,20
4,Narcos: Mexico,Crime Drama,2,20
...,...,...,...,...
553,The Last Narc,Drug Documentary,1,4
554,All or Nothing: Tottenham Hotspur,Sports Documentary,1,9
555,Fernando,Sports Documentary,1,5
556,El Desafío: ETA,Docuseries,1,8


In [94]:
#set db and connections
rds_connection_string = "{}:{}@localhost:5432/netflix_TVshows".format(username,password)
engine = create_engine(f'postgresql://{rds_connection_string}')

In [95]:
#check tables in dbs
engine.table_names()

['shows_detail', 'shows_other']

In [96]:
#convert df1 into corresponded table in db
df1.to_sql(name='shows_detail',con=engine,if_exists='append',index=False)


In [97]:
#convert df2 into corresponded table in db
df2.to_sql(name='shows_other',con=engine, if_exists='append',index=False)

In [100]:
#check data in db
pd.read_sql_query('select * from shows_detail',con=engine).head()

,title,country,publish_date,release_year,description
0,3%,Brazil,"August 14, 2020",2020,In a future where the elite inhabit an island ...
1,46,Turkey,"July 1, 2017",2016,A genetics professor experiments with a treatm...
2,1983,"Poland, United States","November 30, 2018",2018,"In this dark alt-history thriller, a naïve law..."
3,1994,Mexico,"May 17, 2019",2019,Archival video and new interviews examine Mexi...
4,​SAINT SEIYA: Knights of the Zodiac,Japan,"January 23, 2020",2020,Seiya and the Knights of the Zodiac rise again...


In [101]:
#check data in db
pd.read_sql_query('select * from shows_other',con=engine).head()

,title,genre,no_of_seasons,no_of_episodes
0,Stranger Things,Science Fiction Horror,3,25
1,The Crown,Historical Drama,4,40
2,Ozark,Crime Drama,3,30
3,Lost in Space,Science Fiction,2,20
4,Narcos: Mexico,Crime Drama,2,20
